In [62]:
SHOW_OUTPUTS = True
LOWER_VALUE = 10000

def ShowTableau(c, a, b, certificate, optimal):
    print("\n")
    print("c[]: {}".format(c))
    print("A[] : {}".format(a))
    print("b[] : {}".format(b))
    print("certificate[] : {}".format(certificate))
    print("Optimal : {}".format(optimal))
    print("\n")

def Identity(n):
    m=[[0 for x in range(n)] for y in range(n)]
    for i in range(0,n):
        m[i][i] = 1
    return m

def Pivotate(which_row, which_column, c, a, b, certificate, operations, optimal):
    
    pivot = a[which_row][which_column]
    if(SHOW_OUTPUTS): print("Pivot: {}".format(pivot))
    
    if(SHOW_OUTPUTS): print("Iterate A[{},{}] over the pivot: {}".format(which_row, which_column ,pivot))
    for i in range(len(a[which_row])):
        if(a[which_row][i] != 0):
            a[which_row][i] = a[which_row][i]/pivot
    if(SHOW_OUTPUTS): print(a)
    
    if(SHOW_OUTPUTS): print("Iterate b[{}] over the pivot: {}".format(which_row, pivot))
    if(b[which_row] != 0):
        b[which_row][0] = b[which_row][0]/pivot
     
    if(SHOW_OUTPUTS): print(b)
    
    if(SHOW_OUTPUTS): print("Iterate operations[{},{}] over the pivot: {}".format(which_row, which_column ,pivot))
    for i in range(len(operations[which_row])):
        if(operations[which_row][i] != 0):
            operations[which_row][i] = operations[which_row][i]/pivot
            
    if(SHOW_OUTPUTS): print(operations)
        
    value = (c[which_column]/a[which_row][which_column]) * -1
    if(SHOW_OUTPUTS): print("The value: {} will be use to set 0 on c[{}] -> {}".format(value, which_column, c[which_column]))
    for i in range(len(a[which_row])):
        if(a[which_row][i] != 0):
            c[i] += a[which_row][i] * value
    
    optimal += b[which_row][0] * value
    
    for i in range(len(certificate)):
        certificate[i] += operations[which_row][i] * value
        
    for i in range(len(a)):
        if(i != which_row):
            if(a[i][which_column] != 0):
                value = (a[i][which_column]/a[which_row][which_column]) * -1
                
                for j in range(len(a[0])):
                    a[i][j] += (a[which_row][j] * value)
            
                b[i][0] += (b[which_row][0] * value)
                    
    return(c, a, b, certificate, operations, optimal)
    

def ConvertToStandardForm(r, v, c, a):
    certificate = [0] * r
    operations = Identity(r)
    gap_vars = Identity(r)
    
    for i in range(len(a)):
        a[i] = a[i] + gap_vars[i] 

    c = c + [0] * r
    
    for i in range(len(c)):
        c[i] = c[i] * -1
        
    return (certificate, operations, a, c)
    
def Primal(c, a, b, certificate, operations, optimal):
    if(SHOW_OUTPUTS): print("Starting primal")
    
    c_index = 0
    while (c_index < len(c)):    
        if(SHOW_OUTPUTS): ShowTableau(c, a, b, certificate, optimal)
        
        if(c[c_index] < 0 ):
            if(SHOW_OUTPUTS): print("Running tableau for c[{}]".format(c_index))
            lower = LOWER_VALUE
            for i in range(len(b)):
                if(b[i][0] != 0):
                    if(SHOW_OUTPUTS): print("Minimizing: {}/{}".format(b[i][0],a[i][c_index]))
                    if(a[i][c_index] != 0 and b[i][0]/a[i][c_index] > 0 and b[i][0]/a[i][c_index] < lower):
                        lower = b[i][0]/a[i][c_index]
                        which_row = i
                        which_column = c_index
                        
            if(lower < 0 or lower == 10000):
                print("ilimitada \n {} \n {}".format(b, certificate))
                break
                
            
            if(SHOW_OUTPUTS): print("Lower value is {} on row #{}".format(lower, which_row))
            c, a, b, certificate, operations, optimal = Pivotate(which_row, which_column, c, a, b, certificate, operations, optimal)
            c_index = 0
        c_index += 1
    
    if(c_index == len(c) and optimal > 0):
        print("otima \n {} \n {} \n {}".format(optimal, b, certificate))
    
    
def Dual(c, a, b, certificate, operations, optimal):   
    if(SHOW_OUTPUTS): print("Starting dual")
    
    if(SHOW_OUTPUTS): ShowTableau(c, a, b, certificate, optimal)
    
    for i in range(len(b)):
        if(b[i][0] < 0):
            b_negative_row = i
            break
            
    if(SHOW_OUTPUTS): print("Negative entry found on b[{}] -> {}".format(b_negative_row, b[b_negative_row][0]))
        
    lower = LOWER_VALUE
    
    for i in range(len(a[b_negative_row])):
        if(c[i] != 0 and a[b_negative_row][i] != 0):
            if(SHOW_OUTPUTS): print("Minimizing: {}/{}".format(c[i],(-1 * a[b_negative_row][i])))
            ## TODO: Verify if a[b_negative_row][i] shold be negative
            if(c[i]/(-1 * a[b_negative_row][i]) < lower):
                which_row = b_negative_row
                which_column = i
                lower = c[i]/(-1 * a[b_negative_row][i])
                
    if(SHOW_OUTPUTS): print("Lower value is {} on a[{},{}]".format(lower, b_negative_row, which_column))
    c, a, b, certificate, operations, optimal = Pivotate(which_row, which_column, c, a, b, certificate, operations, optimal)
    
        
def SimplexTableau(r, v, c, a, b):
    certificate, operations, a , c = ConvertToStandardForm(r, v, c, a)
    optimal = 0
    
    for i in range(len(b)):
        if(b[i][0] < 0):
            Dual(c, a, b, certificate, operations, optimal)
            return
    Primal(c, a, b, certificate, operations, optimal)    

In [63]:
r = 3
v = 3
c = [-4, -8, -9]
a = [[2, -1, 5],[3, -4, 1],[-1, 0, -2]]
b = [[1],[3],[-8]]
SimplexTableau(r,v,c,a,b)

Starting dual


c[]: [4, 8, 9, 0, 0, 0]
A[] : [[2, -1, 5, 1, 0, 0], [3, -4, 1, 0, 1, 0], [-1, 0, -2, 0, 0, 1]]
b[] : [[1], [3], [-8]]
certificate[] : [0, 0, 0]
Optimal : 0


Negative entry found on b[2] -> -8
Minimizing: 4/1
Minimizing: 9/2
Lower value is 4.0 on a[2,0]
Pivot: -1
Iterate A[2,0] over the pivot: -1
[[2, -1, 5, 1, 0, 0], [3, -4, 1, 0, 1, 0], [1.0, 0, 2.0, 0, 0, -1.0]]
Iterate b[2] over the pivot: -1
[[1], [3], [8.0]]
Iterate operations[2,0] over the pivot: -1
[[1, 0, 0], [0, 1, 0], [0, 0, -1.0]]
The value: -4.0 will be use to set 0 on c[0] -> 4


In [52]:
r = 4 
v = 3
c = [1, 1, 1]
a = [[1, 0, 0],[0, 1, 0],[0, 0, 1],[1, 1, 1]]
b = [[-1],[-1],[-1],[-1]]
SimplexTableau(r,v,c,a,b)

Starting dual


c[]: [-1, -1, -1, 0, 0, 0, 0]
A[] : [[1, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 1, 0], [1, 1, 1, 0, 0, 0, 1]]
b[] : [[-1], [-1], [-1], [-1]]
certificate[] : [0, 0, 0, 0]
Optimal : 0


Negative entry found on b[0] -> -1
Minimizing: -1/-1


In [25]:
r = 3
v = 3
c = [3,2,4]
a = [[1,1,2],[2,0,3],[2,1,3]]
b = [[4],[5],[7]]
SimplexTableau(r,v,c,a,b)

Starting primal


c[]: [-3, -2, -4, 0, 0, 0]
A[] : [[1, 1, 2, 1, 0, 0], [2, 0, 3, 0, 1, 0], [2, 1, 3, 0, 0, 1]]
b[] : [[4], [5], [7]]
certificate[] : [0, 0, 0]
Optimal : 0


Running tableau for c[0]
Minimizing: 4/1
Minimizing: 5/2
Minimizing: 7/2
Lower value is 2.5 on row #1
Pivot: 2
Iterate A[1,0] over the pivot: 2
[[1, 1, 2, 1, 0, 0], [1.0, 0, 1.5, 0, 0.5, 0], [2, 1, 3, 0, 0, 1]]
Iterate b[1] over the pivot: 2
[[4], [2.5], [7]]
Iterate operations[1,0] over the pivot: 2
[[1, 0, 0], [0, 0.5, 0], [0, 0, 1]]
The value: 3.0 will be use to set 0 on c[0] -> -3


c[]: [0.0, -2, 0.5, 0, 1.5, 0]
A[] : [[0.0, 1.0, 0.5, 1.0, -0.5, 0.0], [1.0, 0, 1.5, 0, 0.5, 0], [0.0, 1.0, 0.0, 0.0, -1.0, 1.0]]
b[] : [[1.5], [2.5], [2.0]]
certificate[] : [0.0, 1.5, 0.0]
Optimal : 7.5


Running tableau for c[1]
Minimizing: 1.5/1.0
Minimizing: 2.5/0
Minimizing: 2.0/1.0
Lower value is 1.5 on row #0
Pivot: 1.0
Iterate A[0,1] over the pivot: 1.0
[[0.0, 1.0, 0.5, 1.0, -0.5, 0.0], [1.0, 0, 1.5, 0, 0.5, 0], [0.0, 1.0, 0

In [5]:
r = 3
v = 3
c = [2,4,8]
a = [[1,0,0],[0,1,0],[0,0,1]]
b = [[1],[1],[1]]
SimplexTableau(r,v,c,a,b)

otima 
 14.0 
 [[1.0], [1.0], [1.0]] 
 [2.0, 4.0, 8.0]


In [6]:
r = 2 
v = 3
c = [1, 0, 0]
a = [[-1, 1, 0,],[-1, 0, 1]] 
b = [[5],[7]]
SimplexTableau(r,v,c,a,b)

ilimitada 
 [[5], [7]] 
 [0, 0]
